# Wrangling OpenStreetMap Data with SQL

# Map Area

San Diego, California,USA

https://www.openstreetmap.org/relation/253832

https://mapzen.com/data/metro-extracts/metro/san-diego_california/


# Downloading OSM XML file and taking a sample out of it

In [5]:
# OSM XML file and taking a sample out of it

import xml.etree.ElementTree as ET
import pprint
from collections import defaultdict
import re
import csv
import codecs
import cerberus
#import schema
import sqlite3


OSM_FILE = "san-diego_california.osm"
SAMPLE_FILE = "sample.osm"
k = 40

# Getting the elements with tags - nodes and way

In [3]:
# Getting the elements with tags - nodes and way

def get_element(filename, tags=('node', 'way', 'relation')):
    context = iter(ET.iterparse(filename, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

# Taking sample from OSM_FILE

In [6]:


# Taking sample from OSM_FILE


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

# Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')
    output.close()
    


# Auditing :

# Counting the element tags in the file

In [3]:
# Counting the element tags in the file

def count_tags(filename):
    tree=ET.iterparse(filename)
    tags={}
    for event,elem in tree:
        if elem.tag not in tags.keys():
            tags[elem.tag]=1
        else:
            tags[elem.tag] = tags[elem.tag]+1
    return tags    
    
with open(OSM_FILE,'rb') as f:
    tags=count_tags(OSM_FILE)
    pprint.pprint(tags)
f.close()

{'bounds': 1,
 'member': 13043,
 'nd': 800964,
 'node': 1017755,
 'osm': 1,
 'relation': 691,
 'tag': 2628756,
 'way': 88531}


# Finding out formatting scheme of K attribute in tags

In [5]:
# Finding out formatting scheme of K attribute in tags

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] = keys['problemchars'] + 1
        else:    
            keys['other'] += 1  
#            print element.attrib['k']
#            print element.attrib['v']
    return keys


def process_keys_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

with open(OSM_FILE,'rb') as f:
    keys = process_keys_map(OSM_FILE)
    pprint.pprint(keys)
f.close()    

{'lower': 637400, 'lower_colon': 1953885, 'other': 37466, 'problemchars': 5}


# Finding unique k (tag attrib['k']) and count

In [7]:
# Finding unique k (tag attrib['k']) and count

def unique_keys(filename):
    distinct_keys=[]
    count=1

    EL=get_element(filename, tags=('node', 'way', 'relation'))
    for element in EL:
        if element.tag=='node' or element.tag=='way':
            for tag in element.iter('tag'):
                if tag.attrib['k'] not in distinct_keys:
                    distinct_keys.append(tag.attrib['k'])
                    count+=1
    distinct_keys.sort()
    print "Total number of unique keys (tag attrib['k'])is {}".format(count)
    
#    return distinct_keys
      
    pprint.pprint(distinct_keys)
    
                
unique_keys(SAMPLE_FILE)  # Using Sample file as input to audit the addr:street key

    

Total number of unique keys (tag attrib['k'])is 202
['FIXME',
 'NHS',
 'Tiger:MTFCC',
 'access',
 'addr:city',
 'addr:country',
 'addr:housenumber',
 'addr:postcode',
 'addr:state',
 'addr:street',
 'addr:unit',
 'aeroway',
 'amenity',
 'area',
 'barrier',
 'beacon',
 'beacon:colour',
 'bench',
 'bicycle',
 'bridge',
 'building',
 'building:levels',
 'building_type',
 'cables',
 'change:lanes',
 'change:lanes:backward',
 'change:lanes:forward',
 'clothes',
 'covered',
 'craft',
 'created_by',
 'crossing',
 'cuisine',
 'cycleway',
 'denomination',
 'description',
 'designation',
 'destination',
 'destination:ref',
 'destination:street',
 'diet:vegan',
 'ele',
 'electrified',
 'emergency',
 'enterance',
 'entrance',
 'exit_to',
 'fee',
 'fire_hydrant:type',
 'fixme',
 'foot',
 'footway',
 'frequency',
 'gnis:Class',
 'gnis:County',
 'gnis:County_num',
 'gnis:ST_alpha',
 'gnis:ST_num',
 'gnis:county_id',
 'gnis:county_name',
 'gnis:created',
 'gnis:feature_id',
 'gnis:id',
 'gnis:import_u

# Finding values(tag attrib['v]) for unique k (tag attrib['k]) and making observation about the data

In [7]:
#Finding values(tag attrib['v]) for unique k (tag attrib['k]) and making observation about the data

def values_for_unique_keys(filename):

        '''
        # Manually provide the item_name value from the list of distinct_keys to calculate 
        # the values for the corresponding unique key value. We would initialize the key 
        # variable with one value at a time and without iterating so that we could have an idea
        # of what sort of values are there for corresponding key value. Also, we would not iterate
        # as it would a long amount of time to calculate the values for all the corresponding unique
        # key value
        '''
        
        key='addr:street'
        values=[]
        EL=get_element(filename, tags=('node', 'way', 'relation'))
        for element in EL:
            for tag in element.iter('tag'):
                if tag.attrib['k']==key:
                    values.append(tag.attrib['v'])
            element.clear()
        print key
        pprint.pprint(values)

        '''
        Using Sample file as input to audit the addr:street key
        '''
values_for_unique_keys(SAMPLE_FILE)  # Using Sample file as input to audit the addr:street key


                    

addr:street
['West Broadway',
 '34th Street',
 'University Avenue',
 'Otay Mesa Road',
 'Balboa Avenue',
 'Columbia Street',
 'N Avenue',
 'Carnegie Street',
 'East Manor Drive',
 'Lawndale Road',
 'Alaquinas Drive',
 'Pomona Avenue',
 'Oliver Avenue',
 'Proctor Place',
 'Cheames Way',
 'Edgeware Road',
 'D Avenue',
 'Thermal Avenue',
 'Ferndale Street',
 'Moorpark Street',
 'Blue Lake Drive',
 'Longwood Street',
 'Kendall Street',
 'Greenford Drive',
 'Crossroads Street',
 'Pepper Glen Way',
 'Challenger Circle',
 'Garston Street',
 'Collegio Drive',
 'Hilltop Drive',
 'Glasgow Drive',
 'Cape May Avenue',
 'Gem Tree Way',
 'Remington Road',
 'Harbin Place',
 'Orozco Road',
 'Adams Avenue',
 'F Avenue',
 '46th Street',
 'Calle del Amor',
 'Subol Court',
 'Helix Street',
 'Balboa Vista Drive',
 'Willow Street',
 'Felton Street',
 'Pacific Avenue',
 'Hemingway Avenue',
 'Catalina Place',
 'Naranca Avenue',
 'Lake Murray Boulevard',
 'Thorn Street',
 'Manhasset Drive',
 'North 1st Street'

# Getting users and count

In [8]:
# Getting users and count

def get_user(element):
    return element.get('user')


def process_users_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.get('user'):
            users.add(get_user(element))
        element.clear()    
    return users


with open(OSM_FILE,'rb') as f:
    users = process_users_map(OSM_FILE)

print len(users)
#pprint.pprint(users)
f.close()

1006


# Problem encountered**  - 1

1) Street address abbreviation - The main problem we encountered in this dataset come from the street name abbreviation inconsistency. In this following code, we build the regex matching the last element in the string, where usually the street type is based. Then we come up with a list of mapping that need not to be cleaned. See Auditing Street Names. 

audit_street_type function search the input string for the regex. If there is a match and it is not within the "expected" list, add the match as a key and add the string to the set.

is_street_name function looks at the attribute k if k="addre:street"

audit function will return the list that match previous two functions. After that, we can do a pretty print the output of the audit. With the list of all the abbreviated street types we can understand and fill-up our "mapping" dictionary as a preparation to convert these street name into proper form.

function update_name is the last step of the process, which take the old street name and update them with a better street name.

# Auditing Street Names

In [11]:
# Auditing Street Names

'''
We create a regex for the street names and store it in street_type_re. 
Furthermore we create a default dictionary that will include sets of different street names.
Then, we will audit the datafile and look for street names that have an ending that is different to
the values in the expected list.

'''



street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Freeway","Circle","Strand","Sterling","Way","Highway",
            "Terrace","South","East","West","North"]

# THIS VARIABLE CONTAINS THE CORRECTIONS
mapping = {
            " St ": " Street ",
            " St.": " Street ",
            " Rd.": " Road ",
            " Rd ": " Road ",
            " Rd": " Road ",
            " Ave ": " Avenue ", 
            " Ave.": " Avenue ",
            " Av ": " Avenue ", 
            " Dr ": " Drive ",
            " Dr.": " Drive",
            " Blvd ": " Boulevard ",
            " Blvd": " Boulevard",
            " Blvd.": " Boulevard",
            " Ct ": " Centre ",
            " Ctr": " Centre",
            " Pl ": " Place ",
            " Ln ": " Lane ",
            " Cir ": " Circle ",
            " Wy": " Way ",
            " S ": " South ",
            " E ": " East ",
            " W ": " West ",
            " N ": "North"
}

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(filename):
    f = open(filename, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(filename, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
            elem.clear()        
    f.close()
    return street_types


def update_name(name, mapping):
    for key,value in mapping.iteritems():
        if key in name:
            return name.replace(key,value)
    return name        
'''
Using the SAMPLE_FILE as an input to audit the street name
'''


st_types = audit(SAMPLE_FILE)

#pprint.pprint(dict(st_types))
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name

       









Corte Playa Cartagena => Corte Playa Cartagena
Port Dunbar => Port Dunbar
Plaza Miraleste => Plaza Miraleste
Corte Playa Pacifica => Corte Playa Pacifica
Avenida Circo => Avenida Circo
Camino Entrada => Camino Entrada
Caminito Quintero => Caminito Quintero
Avenida Benjamin => Avenida Benjamin
Calle Santiago => Calle Santiago
Calle de la Plata => Calle de la Plata
Camino de Las Palmas => Camino de Las Palmas
Caminito Marcial => Caminito Marcial
Emerald Cove => Emerald Cove
Via Bartolo => Via Bartolo
Caminito Amparo => Caminito Amparo
Caminito Joven => Caminito Joven
Via Palma => Via Palma
Paseo Ladera => Paseo Ladera
Plaza Palo Alto => Plaza Palo Alto
Rue Michelle => Rue Michelle
Camino Elevado => Camino Elevado
Caminito Pacheco => Caminito Pacheco
Plaza del Cid => Plaza del Cid
Rue Cheaumont => Rue Cheaumont
Rue Michael => Rue Michael
Casa Alta => Casa Alta
Plaza Toluca => Plaza Toluca
El Paseo Grande => El Paseo Grande
Caminito Agrado => Caminito Agrado
Caminito Halago => Caminito Hal

In the above audit of the street name, SAMPLE_FILE was used and there were no potential errors produced.However, errors were produced with bigger SAMPLE_FILE and original OSM file which were then corrected.

# Improving Street Names - Cleaning up and Fixing the Street Names during shape_element function

In [78]:
# Cleaning up and Fixing the Street Names

'''
Now we are going to do some data cleaning to enhance the data quality of the street names.
We have identified sets of street name endings that have been expected. Through a a mapping dictionary we 
indicate the desired changes. We do this for the street name endings (mapping).
'''




street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons","Freeway","Circle","Strand","Sterling","Way","Highway",
            "Terrace","South","East","West","North"]

# UPDATE THIS VARIABLE
mapping = {
            " St ": " Street ",
            " St.": " Street ",
            " Rd.": " Road ",
            " Rd ": " Road ",
            " Rd": " Road ",
            " Ave ": " Avenue ", 
            " Ave.": " Avenue ",
            " Av ": " Avenue ", 
            " Dr ": " Drive ",
            " Dr.": " Drive",
            " Blvd ": " Boulevard ",
            " Blvd": " Boulevard",
            " Blvd.": " Boulevard",
            " Ct ": " Centre ",
            " Ctr": " Centre",
            " Pl ": " Place ",
            " Ln ": " Lane ",
            " Cir ": " Circle ",
            " Wy": " Way ",
            " S ": " South ",
            " E ": " East ",
            " W ": " West ",
            " N ": "North"
}


'''
The update name function implements the change. If a street name has the defined string which is defined in the mapping
dictionary, then the change is made as defined.
'''

'''
Below 2 functions would be used during shape_element function execution to formatt the street name

'''

def update_street_name(name, mapping):
    for key,value in mapping.iteritems():
        if key in name:
            return name.replace(key,value)
    return name        

def audit_street_name_tag(element): 
    street_name=element.get('v')
    m = street_type_re.search(street_name)
    if m:
        better_street_name=update_street_name(street_name,mapping)
        return better_street_name
    return street_name
              

# Problem encountered**  - 2

2) Postcodes - We can re-use part of the code in street abbreviation problem and briefly modify it to use it here. Although most of the postcode is correct, there're still a lot of  postcode with incorrect 5 digit formats. Like some postcodes have "-" followed by another string of numbers, some postcodes have "CA " attached infront of them, some postcodes are more than 5 digits. All of these cases have been dealt to produce a clean postcode.

The output of the clean postcode is summarised below. 

# Auditing Postal Codes  

In [10]:
# Auditing Postal Codes

'''
In this Section we are going to audit postal codes to check for potential errors. This is a very similar process compared to
to our cleaning street name strategy

'''

zip_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

zip_types = defaultdict(set)

expected_zip = {}

def audit_zip_codes(zip_types, zip_name, regex, expected_zip):
    m = regex.search(zip_name)
    if m:
        zip_type = m.group()
        if zip_type not in expected_zip:
             zip_types[zip_type].add(zip_name)

def is_zip_name(elem):
    return (elem.attrib['k'] == "addr:postcode")


def audit(filename, regex):
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag == "way" or elem.tag == "node":
            for tag in elem.iter("tag"):
                if is_zip_name(tag):
                    audit_zip_codes(zip_types, tag.attrib['v'], regex, expected_zip)
    pprint.pprint(dict(zip_types))


'''
Using the SAMPLE_FILE as an input to audit the postcodes
'''   
    
audit(SAMPLE_FILE, zip_type_re)


for zip_type, ways in zip_types.iteritems(): 
        for name in ways:
            if "-" in name:
                name = name.split("-")[0].strip()
            if "CA " in name:
                name = name.split("CA ")[1].strip('CA ')
            elif len(str(name))>5:
                name=name[0:5]
            elif name.isdigit()==False:
                print 'OK'
            print name  



 
                
        

{'91902': set(['91902']),
 '91910': set(['91910']),
 '91911': set(['91911']),
 '91913': set(['91913']),
 '91914': set(['91914']),
 '91915': set(['91915']),
 '91932': set(['91932']),
 '91941': set(['91941']),
 '91942': set(['91942']),
 '91945': set(['91945']),
 '91950': set(['91950']),
 '91977': set(['91977']),
 '91978': set(['91978']),
 '92019': set(['92019']),
 '92020': set(['92020']),
 '92021': set(['92021']),
 '92037': set(['92037']),
 '92040': set(['92040']),
 '92071': set(['92071']),
 '92101': set(['92101']),
 '92102': set(['92102']),
 '92103': set(['92103']),
 '92104': set(['92104']),
 '92105': set(['92105']),
 '92106': set(['92106']),
 '92107': set(['92107']),
 '92108': set(['92108']),
 '92109': set(['92109']),
 '92110': set(['92110']),
 '92111': set(['92111']),
 '92113': set(['92113']),
 '92114': set(['92114']),
 '92115': set(['92115']),
 '92116': set(['92116']),
 '92117': set(['92117']),
 '92118': set(['92118']),
 '92119': set(['92119']),
 '92120': set(['92120']),
 '92121': se

In the above audit of the postcode, SAMPLE_FILE was used and there were no potential errors produced.However, errors were produced with bigger SAMPLE_FILE and original OSM file which were then corrected.

# Cleaning and Fixing Postal Codes during the shape_element function

In [81]:
'''
We want to have all postal codes in the standard 5 digit display. This means we have to change the postal codes 
that have more than 5 digits, the ones that beginn with "CA" and any other ones that differ from the the plain 5
digit display.
'''


'''
Below 2 functions would be used during shape_element function execution to formatt the postcode
'''


def update_postcode(name): 
    if "-" in name:
        name = name.split("-")[0].strip()
    elif "CA" in name:
        name = name.split("CA ")[1].strip('CA ')
    elif len(str(name))>5:
        name=name[0:5]
    elif name.isdigit()==False:
         name=00000
    return name



def audit_postcode_tag(element,regex=re.compile(r'\b\S+\.?$', re.IGNORECASE)):
    post_code=element.get('v')
    m = regex.search(post_code)
    if m:
        better_postcode=update_postcode(post_code)
        return better_postcode
    return post_code
        

# Preparing for Database - SQL

In [82]:
 '''
 Note: The schema is stored in a .py file in order to take advantage of the
 int() and float() type coercion functions. Otherwise it could easily stored as
 as JSON or another serialized format.
 '''

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

# Defining CSV Files and their respective columns

In [14]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


SCHEMA = schema


NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


# Shaping up the element

In [15]:
 """Clean and shape node or way XML element to Python dict"""

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = [] # Handle secondary tags the same way for both node and way elements
             

   

    if element.tag=='node':
        for field in node_attr_fields:
            node_attribs[field]=element.get(field)
                 
        if element.find('tag') is None:
            pass
#           print 'No Tags'
           
        elif element.find('tag') is not None:
            tag_attrib={}
            node_tag_fields=NODE_TAGS_FIELDS
            for tag in element.iter('tag'):
                if PROBLEMCHARS.search(tag.attrib['k']):
                    pass
                elif LOWER_COLON.search(tag.attrib['k']):
                    tag_attrib[node_tag_fields[0]]=element.get('id')
                    tag_attrib[node_tag_fields[1]]=tag.get('k')[(tag.get('k').find(':')+1):]
                    if tag.attrib['k']== "addr:street":
                        tag_attrib[node_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        tag_attrib[node_tag_fields[2]]=audit_postcode_tag(tag)       
                    else:
                        tag_attrib[node_tag_fields[2]]=tag.get('v')
                    tag_attrib[node_tag_fields[3]]=tag.get('k').split(':')[0]
                    tags.append(tag_attrib.copy())
                
                else:
                    tag_attrib[node_tag_fields[0]]=element.get('id')
                    tag_attrib[node_tag_fields[1]]=tag.get('k')
                    if tag.attrib['k']== "addr:street":
                        tag_attrib[node_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        tag_attrib[node_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:    
                        tag_attrib[node_tag_fields[2]]=tag.get('v')
                    tag_attrib[node_tag_fields[3]]=default_tag_type
                    tags.append(tag_attrib.copy())
            
#        pprint.pprint( {'node':node_attribs,'node_tags':tags})  
        
                
    elif element.tag=='way':
        for field in way_attr_fields:
            way_attribs[field]=element.get(field)
    
        way_node_attrib={}
        way_node_fields=WAY_NODES_FIELDS
        for nd in element.findall('nd'):
            way_node_attrib[way_node_fields[0]]=element.get('id')
            way_node_attrib[way_node_fields[1]]=nd.get('ref')
            way_node_attrib[way_node_fields[2]]=element.findall('nd').index(nd)
            way_nodes.append(way_node_attrib.copy())
#       pprint.pprint({'way':way_attribs,'way_nodes':way_nodes})
        
        
        
        
        if element.find('tag') is None:
            pass
#           print 'No Tags'
           
        elif element.find('tag') is not None:
            way_tag_attrib={}
            way_tag_fields=WAY_TAGS_FIELDS
            for tag in element.iter('tag'):
                if PROBLEMCHARS.search(tag.attrib['k']):
                    pass
                elif LOWER_COLON.search(tag.attrib['k']):
                    way_tag_attrib[way_tag_fields[0]]=element.get('id')
                    way_tag_attrib[way_tag_fields[1]]=tag.get('k')[(tag.get('k').find(':')+1):]
                    if tag.attrib['k']== "addr:street":
                        way_tag_attrib[way_tag_fields[2]]=audit_street_name_tag(tag)
                    elif tag.attrib['k']== "addr:postcode":
                        way_tag_attrib[way_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:
                        way_tag_attrib[way_tag_fields[2]]=tag.get('v')
                    way_tag_attrib[way_tag_fields[3]]=tag.get('k').split(':')[0]
                    tags.append(way_tag_attrib.copy())
                    
                else:
                    way_tag_attrib[way_tag_fields[0]]=element.get('id')
                    way_tag_attrib[way_tag_fields[1]]=tag.get('k')
                    if tag.attrib['k']== "addr:street":
                        way_tag_attrib[way_tag_fields[2]]=audit_street_name_tag(tag) 
                    elif tag.attrib['k']== "addr:postcode":
                        way_tag_attrib[way_tag_fields[2]]=audit_postcode_tag(tag)    
                    else:   
                        way_tag_attrib[way_tag_fields[2]]=tag.get('v')
                    way_tag_attrib[way_tag_fields[3]]=default_tag_type
                    tags.append(way_tag_attrib.copy())
#        pprint.pprint({'way':way_attribs,'way_tags':tags})
#        pprint.pprint({'way':way_attribs,'way_nodes':way_nodes,'way_tags':tags})
        
    

    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
  



     

# Helper function - Validating the element

In [16]:
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

# Helper function - UnicodeDictWriter

In [17]:
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

# Writing CSV Files

In [18]:
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

# Loading the data into CSV file from OSM File

In [39]:
process_map(OSM_FILE, validate=True)

# Creating and Connecting to SQL database 

In [40]:
db = sqlite3.connect("san-diego_california")  # Connect to the database
                                              # "san-diego_california" is the name of the sqlite database file
                                
c = db.cursor() # Get a cursor object


# Creating nodes table

In [41]:
# creating nodes table in database san-diego_california and inserting values into table nodes

query="DROP TABLE IF EXISTS nodes;" # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE nodes (id INTEGER PRIMARY KEY NOT NULL,lat REAL,lon REAL,user TEXT,uid INTEGER,version INTEGER,changeset INTEGER,timestamp TEXT);"
c.execute(query)
db.commit()

# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('nodes.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['lat'].decode("utf-8"),i['lon'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]
    
# insert the formatted data

c.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
db.commit()
f.close()



# Creating nodes_tags table

In [42]:
# creating nodes_tags table in database san-diego_california and inserting values into table nodes_tags

query="DROP TABLE IF EXISTS nodes_tags;"  # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE nodes_tags (id INTEGER,key TEXT,value TEXT,type TEXT,FOREIGN KEY (id) REFERENCES nodes(id));"
c.execute(query)
db.commit()

# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('nodes_tags.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

# insert the formatted data

c.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
db.commit()
f.close()


# Creating ways table

In [43]:
# creating ways table in database san-diego_california and inserting values into table ways

query="DROP TABLE IF EXISTS ways;"   # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE ways(id INTEGER PRIMARY KEY NOT NULL,user TEXT,uid INTEGER,version TEXT,changeset INTEGER,timestamp TEXT);"
c.execute(query)
db.commit()

# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('ways.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['user'].decode("utf-8"),i['uid'].decode("utf-8"),i['version'].decode("utf-8"),i['changeset'].decode("utf-8"),i['timestamp'].decode("utf-8")) for i in dr]

# insert the formatted data        
    
c.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?);", to_db)
db.commit()
f.close()


# Creating ways_nodes table

In [44]:
# creating ways_nodes table in database san-diego_california and inserting values into table ways_nodes

query="DROP TABLE IF EXISTS ways_nodes;"  # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE ways_nodes (id INTEGER NOT NULL,node_id INTEGER NOT NULL,position INTEGER NOT NULL,FOREIGN KEY (id) REFERENCES ways(id),FOREIGN KEY (node_id) REFERENCES nodes(id));"
c.execute(query)
db.commit()

# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('ways_nodes.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['node_id'].decode("utf-8"),i['position'].decode("utf-8")) for i in dr]

    
# insert the formatted data 
    
c.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?,?,?);", to_db)
db.commit()
f.close()




# Creating ways_tags table

In [45]:
# creating ways_tags table in database san-diego_california and inserting values into table ways_nodes

query="DROP TABLE IF EXISTS ways_tags;"   # Dropping the table if it already exists
c.execute(query);
db.commit()
query = "CREATE TABLE ways_tags (id INTEGER NOT NULL,key TEXT NOT NULL,value TEXT NOT NULL,type TEXT,FOREIGN KEY (id) REFERENCES ways(id));"
c.execute(query)
db.commit()


# Read in the csv file as a dictionary, format the data as a list of tuples:

with open('ways_tags.csv','rb') as f: 
    dr = csv.DictReader(f)
    to_db = [(i['id'].decode("utf-8"),i['key'].decode("utf-8"),i['value'].decode("utf-8"),i['type'].decode("utf-8")) for i in dr]

# insert the formatted data 

c.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?,?,?,?);", to_db)
db.commit()
f.close()



# Overview of The Data

This section contains basic statistics about the San Diego OpenStreetMap dataset and the SQL queries used to gather them.

# File sizes - 

san-diego_california.osm  290 MB                                                                                       
san-diego_california.db   195 MB                                                                                    
nodes.csv                 79.2 MB                                                                                      
nodes_tags.csv            83.0 MB                                                                                     
ways.csv                  4.98 MB                                                                                                                                                                             
ways_nodes.csv            18.3 MB  
ways_tags.csv             19.2 MB                                                                                       

# Number of unique users

In [46]:
query = "SELECT COUNT(DISTINCT(e.uid))FROM (SELECT uid FROM Nodes UNION ALL SELECT uid FROM Ways) as e;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(993,)]


Number of Unique Users is 993.

# Number of nodes

In [62]:
query = "SELECT count(DISTINCT(id)) FROM nodes;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(1017755,)]


Number of Nodes is  1017755.

# Number of Ways

In [63]:
query = "SELECT count(DISTINCT(id)) FROM ways;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(88531,)]


Number of ways are 88531.

# Number of chosen type of nodes

In [49]:
# number of chosen type of nodes

query = "SELECT type , count(*) as num  FROM nodes_tags group by type order by num desc;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)


[(u'addr', 1643780),
 (u'regular', 395218),
 (u'is_in', 45587),
 (u'gnis', 7111),
 (u'seamark', 1790),
 (u'source', 176),
 (u'wetap', 113),
 (u'beacon', 99),
 (u'payment', 64),
 (u'recycling', 53),
 (u'diet', 50),
 (u'name', 38),
 (u'contact', 29),
 (u'prosper', 28),
 (u'fire_hydrant', 24),
 (u'service', 21),
 (u'caltrans', 20),
 (u'tiger', 20),
 (u'maxspeed', 16),
 (u'ref', 16),
 (u'alt_name', 9),
 (u'census', 9),
 (u'exit_to', 8),
 (u'railway', 8),
 (u'tower', 8),
 (u'building', 7),
 (u'healthcare', 5),
 (u'crossing', 4),
 (u'charging_station', 3),
 (u'disused', 3),
 (u'internet_access', 3),
 (u'location', 3),
 (u'sanitary_dump_station', 3),
 (u'socket', 3),
 (u'buoy', 2),
 (u'fuel', 2),
 (u'monitoring', 2),
 (u'note', 2),
 (u'place', 2),
 (u'social_facility', 2),
 (u'toilets', 2),
 (u'topmark', 2),
 (u'traffic_signals', 2),
 (u'wheelchair', 2),
 (u'description', 1),
 (u'golf', 1),
 (u'heritage', 1),
 (u'swing_gate', 1)]


In [94]:
# number of chosen type of nodes, like cafes, shops etc.

query = "SELECT value, count(*) FROM (select key,value from nodes_tags UNION ALL select key,value from ways_tags) as e where value like '%cafe%';"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)


print '\n'

query = "SELECT value, count(*)  FROM (select key,value from nodes_tags UNION ALL select key,value from ways_tags) as e where value like 'shop%';"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)


[(u'cafe', 287)]


[(u'shop', 11)]


# Top 10 contributing users

In [64]:
#Top 10 contributing users

query = "select e.user, count(*) as num from (select user from nodes UNION ALL select user from ways) as e group by user order by num desc limit 10;"
c.execute(query)
rows=c.fetchall()
print 'Top 10 contributing users and their contribution:\n'
pprint.pprint(rows)



#Total users
print '\n'
print 'Total appearances of the users:\n'
query = "select count(e.user) from (select user from nodes UNION ALL select user from ways) as e;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

Top 10 contributing users and their contribution:

[(u'n76', 335145),
 (u'Adam Geitgey', 162980),
 (u'Sat', 128338),
 (u'woodpeck_fixbot', 91625),
 (u'Zian Choy', 16886),
 (u'Brian@Brea', 15927),
 (u'TieFaith', 12997),
 (u'stevea', 12641),
 (u'evil saltine', 12147),
 (u'javbw', 10606)]


Total appearances of the users:

[(1106286,)]


So, from the above 2 queries, it is observed that user 'n76' has appeared 335145 times from 1106286 which consists of about 30.29 % of the total counts.

# Top 20 keys in tags with respect to count size

In [65]:
#Top 20 keys in tags with respect to count size

query = "select e.key , count(*) as num from (select key from nodes_tags UNION ALL select key from ways_tags) as e group by e.key order by num desc limit 20;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(u'source', 386765),
 (u'country', 350731),
 (u'street', 344486),
 (u'housenumber', 344298),
 (u'city', 314944),
 (u'postcode', 307985),
 (u'highway', 67441),
 (u'name', 41278),
 (u'county', 31327),
 (u'cfcc', 31135),
 (u'name_base', 27174),
 (u'name_type', 24607),
 (u'building', 23242),
 (u'zip_left', 22980),
 (u'reviewed', 22796),
 (u'zip_right', 22403),
 (u'tlid', 15593),
 (u'oneway', 14123),
 (u'separated', 14044),
 (u'building_type', 13176)]


From the above query, it is observed that tags with K value 'Source' occured maximum time.

# Top 20 value in tags with respect to count size

In [66]:
#Top 20 value in tags with respect to count size

query = "select e.value, count(e.value) as num from (select value from nodes_tags UNION ALL select value from ways_tags) as e group by value order by num desc limit 20;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(u'US', 350708),
 (u'SanGIS Addresses Public Domain (http://www.sangis.org/)', 342634),
 (u'San Diego', 187987),
 (u'no', 41193),
 (u'yes', 38538),
 (u'San Diego, CA', 31345),
 (u'A41', 25902),
 (u'Chula Vista', 24073),
 (u'El Cajon', 19432),
 (u'residential', 19166),
 (u'La Mesa', 16561),
 (u'92114', 16517),
 (u'92117', 15953),
 (u'tiger_import_dch_v0.6_20070809', 15602),
 (u'91977', 14507),
 (u'service', 14457),
 (u'Spring Valley', 14372),
 (u'92154', 14147),
 (u'91941', 13491),
 (u'CA', 13058)]


From the above query, it is observed that tags with v value 'US' occured maximum time.

# when was the 1st Contribution made and by whom

In [67]:
query = "select e.user, e.timestamp as num from (select user,timestamp from nodes UNION ALL select user,timestamp from ways) as e order by num limit 1;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(u'DaveHansenTiger', u'2007-09-10T17:51:45Z')]


1st Contribution to San-diego OSM database was made by user 'DaveHansenTiger' at 17:56 hrs on 2007-09-10

# Top 20 Website Links

In [68]:
query = "select e.value, count(*) as num from (select value from nodes_tags UNION ALL select value from ways_tags) as e WHERE value LIKE '%www.%' group by e.value order by num desc limit 20;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(u'SanGIS Addresses Public Domain (http://www.sangis.org/)', 342634),
 (u'SanGIS Footprints_Nonresidential_SD public domain (http://www.sangis.org/)',
  12965),
 (u'SanDag Public Transit (http://www.sandag.org/)', 2161),
 (u'SanGIS Business Sites Public Domain (http://www.sangis.org/)', 667),
 (u'Caltrans http://www.dot.ca.gov/hq/traffops/trucks/truckmap/', 537),
 (u'SanGIS Freeways_SD public domain (http://www.sangis.org/)', 145),
 (u'TIGER/Line\xae 2008 Place Shapefiles (http://www.census.gov/geo/www/tiger/)',
  67),
 (u'http://www.mcdonalds.com/', 55),
 (u'http://www.southbayexpressway.com/about/faq.php#miscbicycles', 44),
 (u'SanGIS Addresses Public Domain (http://www.sangis.org/);SanGIS Business Sites Public Domain (http://www.sangis.org/)',
  37),
 (u'Bing;SanDag Public Transit (http://www.sandag.org/)', 35),
 (u'SanGIS Footprints_Military_Facility public domain (http://www.sangis.org/)',
  29),
 (u'http://www.dot.ca.gov/hq/tsip/gis/datalibrary/gisdatalibrary.html', 26),
 (u'htt

# Popular Cuisines

In [69]:
query="select value,count(*) as num from (select key,value from nodes_tags UNION ALL select key,value from ways_tags) as e where e.key like '%cuisine%' group by value order by num desc limit 20;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(u'burger', 205),
 (u'mexican', 128),
 (u'sandwich', 97),
 (u'pizza', 77),
 (u'american', 42),
 (u'coffee_shop', 41),
 (u'chinese', 28),
 (u'italian', 27),
 (u'chicken', 23),
 (u'japanese', 19),
 (u'thai', 18),
 (u'sushi', 16),
 (u'ice_cream', 15),
 (u'bagel', 12),
 (u'indian', 11),
 (u'regional', 10),
 (u'greek', 8),
 (u'steak_house', 6),
 (u'vegan', 6),
 (u'breakfast', 5)]


# List of keys where type is addr

In [70]:
query="select DISTINCT(key) from nodes_tags where type='addr';"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(u'city',),
 (u'street',),
 (u'country',),
 (u'housenumber',),
 (u'state',),
 (u'postcode',),
 (u'unit',),
 (u'housename',),
 (u'full',),
 (u'county',),
 (u'door',),
 (u'zipcode',),
 (u'suite',)]


# Common ammenities

In [71]:
query="select value, count(*) as num from nodes_tags where key='amenity' group by value order by num desc limit 20;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(u'place_of_worship', 913),
 (u'fast_food', 534),
 (u'restaurant', 461),
 (u'school', 299),
 (u'bar', 269),
 (u'cafe', 162),
 (u'fuel', 106),
 (u'bank', 81),
 (u'bench', 71),
 (u'drinking_water', 71),
 (u'library', 65),
 (u'toilets', 65),
 (u'bicycle_parking', 64),
 (u'post_office', 55),
 (u'atm', 53),
 (u'hospital', 51),
 (u'fountain', 48),
 (u'post_box', 47),
 (u'shelter', 47),
 (u'waste_basket', 42)]


# Religions

In [72]:
query="select value, count(*) as num from (select key,value from nodes_tags UNION ALL select key,value from ways_tags) as e  where key='religion' group by value order by num desc;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(u'christian', 874),
 (u'jewish', 9),
 (u'muslim', 7),
 (u'buddhist', 5),
 (u'hindu', 4),
 (u'ascended_master_teachings', 1),
 (u'bahai', 1),
 (u'scientologist', 1),
 (u'taoist', 1),
 (u'unitarian', 1),
 (u'unitarian_universalist', 1)]


# Dataset Improvement - 

# Percentage of wheelchair accessible nodes

In [107]:
query="select count(*) from (select key,value from nodes_tags UNION ALL select key,value from ways_tags) as e  where key like '%wheelchair%';"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(134,)]


# Number of nodes

In [108]:
query="select count(DISTINCT(id)) from  nodes_tags;"
c.execute(query)
rows=c.fetchall()

pprint.pprint(rows)

[(370105,)]


# Percentage of nodes with wheelchair accessibility information

134/370105 = 3.620594155712568e-4 which is almost 0.

Based on the above 2 queries, approximately 0.003% of the nodes in the dataset contain wheelchair accessibility information. That seems like a strikingly low number, even with a large amount of nodes. 

# Ideas for additional improvements 

One way to improve this wheelchair accessibility number is to leverage the public data provided by http://data.sandiego.gov/, http://www.sangis.org/download/), and similar resources.Wheelchair Accessibility information for hundreds of restaurants, cafes, tourist attractions, community centers, and other public spaces could be added to the dataset. Programmatically extracting the yes/no information and adding it to the OpenStreetMap dataset would likely be most efficient. We can also cross-reference/cross-validate missing data from other database like Google API as each node has a particular coordinate (lattitude & longtitude). It's also extremely important to attract more people to improving maps. Use of gamification can be another measure. It's reasonable to establish ranking system like on Kaggle or badge system like on Khan academy. It would increase the productivity and the missing information in the OSM data would be added.

# Anticipated problems in implementing the improvement

1) One difficulty would be dealing with naming inconsistencies between different data sources and nodes already in the OpenStreetMap dataset, though this could be overcome with careful string handling and a human verifying inputted data.

2) Amount of effort to engineer all these processes and the cost of creating, auditing & maintaining these initiatives could be so overwhelm and may require a dedicated team responsible for all these projects.

# Conclusion

The San-Diego OpenStreetMap dataset is a quite large. When we audit the data, it was very clear that although there are minor error caused by human input, the dataset is fairly well-cleaned. Considering there're hundreds of contributors for this map, there are number of human errors in this dataset. I'd recommend a srtuctured input form so everyone can input the same data format to reduce this error or we can create a more robust script to clean the data on a regular basis.While it is clear that the data is not 100% clean, I believe it was sufficiently cleaned for the purposes of this project.A detailed investigation could also be done to identify which users created the features with incorrect or mismatched values in order to potentially identify additional documents for closer scrutiny or to predict what types of users are more prone to producing errors.This dataset shows the need for a standardized data schema, as well as the difficulty of having many different data sources and having information which needs to be constantly updated.